## 0. Instalación y carga de librerías

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import random
import yfinance as yf
import datetime
import plotly.express as px
#!pip install git+https://github.com/fedefliguer/bootstrap_backtester@main#egg=bootstrap_backtester
from bootstrap_backtester import *

## 1. Definición de la estrategia

In [ ]:
strategy = 'e_RSI' # La estrategia debe estar en la función build_individual_strategy(), llevando para cada observación C, M o V según sea Compra, Mantener o Vender.

## 2. Definir listado total de activos disponibles para la estrategia

In [ ]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_tickers = list(sp500.Symbol)
sp500_tickers = [e.replace('.','-') for e in sp500_tickers]

## 3. Definir cantidad de iteraciones y activos por iteración

In [ ]:
numero_iteraciones = int(len(sp500_tickers)/18)
numero_activos_por_iteracion = int(len(sp500_tickers)/12)

## 4. Ejecutar iteraciones y se guardan resultados

In [ ]:
results = pd.DataFrame(columns=['metrica','grupo','valor'])
j = 0
while j < numero_iteraciones:
    j = j + 1
    print('Iteración', j)

    tickers = random.sample(sp500_tickers, numero_activos_por_iteracion)
    empieza = str(random.randint(2004, 2022)) + '-' + str(random.randint(1, 12)) + '-01'
    termina = datetime.datetime.strptime(empieza, "%Y-%m-%d") + datetime.timedelta(days=random.randint(360, 1000))
    
    print(tickers, empieza, termina)
    df_global, daily_returns = descarga(tickers, empieza, termina)
    df_global = build_individual_strategy(df_global, estrategia = strategy)    
    df_trades = build_trades(df_global, estrategia = strategy)

    benchmark_returns = yf.download(tickers, start=empieza, end=termina).iloc[:, 0:len(tickers)].pct_change().mean(axis=1)
    strategy_returns = build_strategy_returns(df_trades, daily_returns, benchmark_returns, duplicados='random', fr = 0)

    strategy_analysis = pd.DataFrame(index = benchmark_returns.index)
    strategy_analysis['benchmark'] = benchmark_returns
    strategy_analysis['strategy'] = strategy_returns
    sinteticos_px = (strategy_analysis+1).cumprod()
    res = resumen(sinteticos_px['strategy'], sinteticos_px['benchmark'])
    results = pd.concat([results, res])

## 5. Boxplot de resultados

In [ ]:
fig = px.box(results, x="metrica", y="valor", color="grupo")
fig.update_traces(quartilemethod="exclusive")
fig.show()